In [2]:
import numpy as np
import pandas as pd

#pip install import_ipynb

import import_ipynb
#from Pedidos import pedidos
#from Fletes import fletes
from Capacidad_de_almacenaje import capacidad_almacenaje
from Clientes_por_planta import clientes_por_planta_
from Capacidad_de_almacenaje import capacidad_almacenaje
from Productos_por_planta import productos_por_planta
from Puertos_por_planta import puertos_por_planta_

# 2da Etapa: Optimización

*Función Objetivo*\
Se desea conocer un conjunto de plantas de almacenaje, puertos de origen y puertos de destino que minimicen el costo total de la cadena de suministro.

Nomenclatura:
- k: ID de pedido
- i: ID de planta de almacenamiento
- p: ID de puerto de origen
- j: ID de puerto de destino
- c: ID de transportista
- s: Nivel de servicio
- t: Tiempo de envío [días]
- m: Modo de transporte (aire o tierra)
- q: Cantidad de items en cada pedido [items]
- w: Peso [kg/item]
- F: Peso máximo [kg/item]

- C: Costo de almacenamiento [USD/item]
- M: Costo fijo de transporte [USD/kg]
- X: Costo de almacenamiento [USD]
- Y: Costo de transporte [USD/kg]
- Z: Costo total de la cadena de suministro [USD]

In [ ]:
pedidos = pedidos[["Producto", "Items", "Peso [kg/item]"]].copy()

df1 = pd.merge(pedidos, productos_por_planta, on = "Producto")
df2 = pd.merge(df1, puertos_por_planta_, on = "Planta")
df3 = pd.merge(df2, fletes, on = ["Puerto de origen"])
df4 = pd.merge(df3, costos_almacenaje, on = "Planta")
df5 = pd.merge(df4, capacidad_almacenaje, on = "Planta")
df6 = pd.merge(df5, clientes_por_planta, on = "Planta")

df6.round(2).head()

*¿La tabla contiene filas duplicadas?*

In [ ]:
df6.duplicated().any()

*Si la tabla tiene filas duplicadas, estas deben ser eliminadas.*

In [ ]:
df6.drop_duplicates(inplace = True)

*¿Cuántas filas y columnas tiene la nueva tabla? (N° filas, N° columnas)*

In [ ]:
df6.shape

*Estadística Descriptiva del Costo de Almacenaje [USD]*

$$ X_{ki} = q_{ki}*C_i $$


In [ ]:
df6["Costo de almacenaje [USD]"] = df6["Items"] * df6["Costo específico [USD/item]"]
df6_agrup = df6.groupby(by = ["Planta"]).agg({"Costo de almacenaje [USD]": ["min", "mean", "max", "sum"]})

total = df6_agrup[("Costo de almacenaje [USD]", "sum")].sum()
df6_agrup[("Costo de almacenaje [USD]", "%")] = df6_agrup[("Costo de almacenaje [USD]", "sum")].apply(lambda x: x / total * 100)

df6_agrup.round(2)

In [ ]:
from functions import plot_bar

plot_bar("Planta", "% Costo de Almacenaje", df6_agrup.index, df6_agrup[("Costo de almacenaje [USD]", "%")])

*Estadística Descriptiva del Costo de Transporte [USD]*

$$ \sum_{k}{w_{kpjcstm}} \leq max \ F_{kpjcstm} $$

$$ Si \ s = CRF \ \Rightarrow \ Y_{kpjcstm} = 0 $$

$$ Si \ s \neq CRF, \ m = Tierra \ \Rightarrow \ Y_{kpjcstm} = \frac{w_{kpjcstm}}{\sum_{k}{w_{kpjcstm}}}*R_{kpjcstm} $$

$$ Si \ s \neq CRF, \ m \neq Tierra \ \Rightarrow \ Y_{kpjcstm} = w_{kpjcstm}*R_{kpjcstm} $$

$$ Si \ Y_{kpjcstm} < M_{kpjcstm} \ \Rightarrow \ Y_{kpjcstm} = M_{kpjcstm} $$

In [ ]:
df6["Peso [kg]"] = df6["Items"] * df6["Peso [kg/item]"]
df6["Peso máximo [kg]"] = df6["Items"] * df6["Peso máximo [kg/item]"]

df6[["Suma de Peso [kg]", "Máximo del Peso máximo [kg]"]] = df6.groupby(by = ["Puerto de origen", "Puerto de destino", "Transportista", 
    "Modo de transporte", "Envío [días]"])["Peso [kg]", "Peso máximo [kg]"].transform("sum", "max").rename(columns = {"Peso [kg]": "Suma de Peso [kg]", 
    "Peso máximo [kg]": "Máximo del Peso máximo [kg]"})

df6["Suma de Peso real [kg]"] = np.where(df6["Suma de Peso [kg]"] <= df6["Peso máximo [kg]"], df6["Suma de Peso [kg]"], df6["Peso máximo [kg]"])
df6.drop(labels = ["Suma de Peso [kg]", "Máximo del Peso máximo [kg]"], axis = 1, inplace = True)

df6.round(2).head()

In [ ]:
a = df6["Peso [kg]"] / df6["Suma de Peso real [kg]"] * df6["Tasa de flete [USD/kg]"]
b = df6["Peso [kg]"] * df6["Tasa de flete [USD/kg]"]
c = df6["Costo mínimo [USD/kg]"]

df6["Costo de transporte [USD]"] = np.where(df6["Nivel de servicio"] == "CRF", 0, 
                                            np.where(df6["Modo de transporte"] == "Tierra", np.where(a > c, a, c),
                                                    np.where(b > c, b, c)))

df6_agrup = df6.groupby(by = ["Puerto de origen", "Puerto de destino"]).agg({"Costo de transporte [USD]": ["min", "mean", "max", "sum"]})

total = df6_agrup[("Costo de transporte [USD]", "sum")].sum()
df6_agrup[("Costo de transporte [USD]", "%")] = df6_agrup[("Costo de transporte [USD]", "sum")].apply(lambda x: x / total * 100)

df6_agrup.round(2)

In [ ]:
plot_bar("Planta", "% Costo de Transporte", df6_agrup.index.levels[0], df6_agrup[("Costo de transporte [USD]", "%")])

*Estadística Descriptiva del Costo Total [USD]*

$$ Z_{ipj} = min \sum_{k}{(X_{ki} + Y_{kpj})} $$

In [11]:
#df6["Costo total [USD]"] = df6["Costo de almacenaje [USD]"] + df6["Costo de transporte [USD]"]
#df6_agrup = df6.groupby(by = ["Planta", "Puerto de origen", "Puerto de destino"]).agg({"Costo total [USD]": ["min", "mean", "max", "sum"]})

#total = df6_agrup[("Costo total [USD]", "sum")].sum()
#df6_agrup[("Costo total [USD]", "%")] = df6_agrup[("Costo total [USD]", "sum")].apply(lambda x: x / total * 100)

#df6_agrup.round(2)